In [45]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_functions
%run My_functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
client = Client()

## Modeling for Working days

In [47]:
# Load datasets
wd_h=dd.read_csv('workingdays_data_prepared.csv' , blocksize = 71e4)
# wd_h['dteday']=pd.to_datetime(wd_h['dteday'], format='%Y-%m-%d')

In [48]:
wd_h.npartitions

8

### Splitting the working days prepared dataset

In [49]:
X_Train = wd_h[
    wd_h["instant"]
    < 15212
].drop(
    ["cnt", "casual", "registered", "dteday",'instant','workingday','holiday'], axis=1
)  
X_Test = wd_h[
    (
        wd_h["instant"]
    >= 15212
    )
].drop(
    ["cnt", "casual", "registered", "dteday",'instant','workingday','holiday'], axis=1
)  ## NONE OF THEM IN DATA
Y_cnt_test = wd_h[
    (
        wd_h["instant"]
    >= 15212
    )
]['cnt']
Y_cnt_train = wd_h[
    wd_h["instant"]
    < 15212
]["cnt"]

### Linear Regression

In [50]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False] ,'normalize':[True,False]}

lm = dcv.GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)
with joblib.parallel_backend('dask'):
    lm.fit(X_Train, Y_cnt_train)
lm.cv_results_
lm_predictions_w = lm.predict(X_Test)
lm.score(X_Test, Y_cnt_test)


0.7925879627114157

### Random Forest

In [51]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = dcv.GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)
with joblib.parallel_backend('dask'):
    rf.fit(X_Train, Y_cnt_train)
rf.cv_results_
rf_predictions_w = rf.predict(X_Test)
rf.score(X_Test, Y_cnt_test)


0.8406789827196025

### XGboost

In [53]:
param_grid = {
    "learning_rate": [0.01, 0.1],
    "max_depth": [4, 8, 12],
    "min_child_weight": [3, 5, 10, 20, 35, 50],
    "subsample": [0.5, 0.75],
    "colsample_bytree": [0.5, 0.75],
    "n_jobs": [-1],
    "n_estimators": [100, 300],
    "random_state": [random_seed],
}
model = xgb.XGBRegressor()

xg = dcv.GridSearchCV(model, param_grid=param_grid, cv=tscv, n_jobs=-1, scoring="r2")
with joblib.parallel_backend("dask"):
    xg.fit(X_Train, Y_cnt_train)
para = xg.best_params_
xg = dxgb.XGBRegressor(
    learning_rate=para["learning_rate"],
    max_depth=para["max_depth"],
    min_child_weight=para["min_child_weight"],
    subsample=para["subsample"],
    colsample_bytree=para["colsample_bytree"],
    n_estimators=para["n_estimators"],
)
with joblib.parallel_backend("dask"):
    xg.fit(X_Train, Y_cnt_train)

xg_predictions_w  = xg.predict(X_Test)
xg.score(X_Test, Y_cnt_test)

0.8910182851771361

In [91]:
pred_w = dd.concat([xg_predictions_w , wd_h[wd_h["instant"] >= 15212]["instant"]] ,  axis = 1)
real_pred_w = dd.concat([Y_cnt_test , wd_h[wd_h["instant"] >= 15212]["instant"]] ,  axis = 1)

## Modeling for Holidays

In [54]:
hd_h=dd.read_csv('holidays_data_prepared.csv' , blocksize = 40e4)

### Splitting the holidays prepared dataset

In [55]:
X_Train_h = hd_h[
    hd_h["instant"]
    < 15212
].drop(
    ["cnt", "casual", "registered", "dteday",'instant','workingday','holiday'], axis=1
)  
X_Test_h = hd_h[
    (
        hd_h["instant"]
    >= 15212
    )
].drop(
    ["cnt", "casual", "registered", "dteday",'instant','workingday','holiday'], axis=1
)  ## NONE OF THEM IN DATA
Y_cnt_test_h = hd_h[
    (
        hd_h["instant"]
    >= 15212
    )
]['cnt']
Y_cnt_train_h = hd_h[
    hd_h["instant"]
    < 15212
]["cnt"]

### Linear Regression

In [56]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False] ,'normalize':[True,False]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)
with joblib.parallel_backend('dask'):
    lm.fit(X_Train_h, Y_cnt_train_h)
lm.cv_results_
lm_predictions_h = lm.predict(X_Test_h)
lm.score(X_Test_h, Y_cnt_test_h)


-3.5671181904079567e+22

### Random Forest

In [57]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)
with joblib.parallel_backend('dask'):
    rf.fit(X_Train_h, Y_cnt_train_h)
rf.cv_results_
rf_predictions_h = rf.predict(X_Test_h)
rf.score(X_Test_h, Y_cnt_test_h)


0.8251328564266366

### XGboost

In [59]:
param_grid = {
    "learning_rate": [0.01, 0.1],
    "max_depth": [4, 8, 12],
    "min_child_weight": [3, 5, 10, 20, 35, 50],
    "subsample": [0.5, 0.75],
    "colsample_bytree": [0.5, 0.75],
    "n_jobs": [-1],
    "n_estimators": [100, 300],
    "random_state": [random_seed],
}
model = xgb.XGBRegressor()

xg = dcv.GridSearchCV(model, param_grid=param_grid, cv=tscv, n_jobs=-1, scoring="r2")
with joblib.parallel_backend("dask"):
    xg.fit(X_Train_h, Y_cnt_train_h)
    
para = xg.best_params_

xg = dxgb.XGBRegressor(
    learning_rate=para["learning_rate"],
    max_depth=para["max_depth"],
    min_child_weight=para["min_child_weight"],
    subsample=para["subsample"],
    colsample_bytree=para["colsample_bytree"],
    n_estimators=para["n_estimators"],
)
with joblib.parallel_backend("dask"):
    xg.fit(X_Train_h, Y_cnt_train_h)

xg_predictions_h  = xg.predict(X_Test_h)
xg.score(X_Test_h, Y_cnt_test_h)

0.8758574001887438

In [106]:
pred_h = dd.concat([xg_predictions_h , hd_h[hd_h["instant"] >= 15212]["instant"]] ,  axis = 1)
real_pred_h = dd.concat([Y_cnt_test_h , hd_h[hd_h["instant"] >= 15212]["instant"]] ,  axis = 1)

### Final score after concatinating the 2 results

In [109]:
pred = pred_h.append(pred_w)
real_pred = real_pred_h.append(real_pred_w)
print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(real_pred['cnt'], pred['predictions'])))
print("Variance score (R2): {:.2f}".format(r2_score(real_pred['cnt'], pred['predictions'])))

Mean squared error (MSE): 4575.23
Variance score (R2): 0.89
